# Notebook E-tivity 3 CE4021 Task 2

Student name: Yvonne Ryan

Student ID: 21208298

<hr style=\"border:2px solid gray\"> </hr>

## Imports

In [ ]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style=\"border:2px solid gray\"> </hr>

## Task 2

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails. You may add as many cells as you require to complete the task.

In [2]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

### Solution

*Notes to self:*

Hypothesis = 'This new email is spam'

Evidence = 'There are words from previous_spam in this email'

| Symbol | Description |
|:-|:-|
| P(H) | Marginal probability that a new email is spam regardless of contents = Proportion of all emails that are spam |
| P(E) | Marginal probability that there are spam words in any given email = Proportion of all emails that contain spam words |
| P(E/H) | Probability that there are spam words in an email given that it is spam |
| P(H/E) | Probability that an email is spam given that it contains spam words |

Bayes' Theorem:
$$ P(H|E) = \frac{P(E|H)P(H)}{P(E)} = \frac{P(E|H)P(H)}{P(E|H)P(H) + P(E|\overline{H})P(\overline{H})} $$

Naive Bayes:
$$ P(S|x_1, x_2, ..., x_n) \approx \frac{P(S)\prod_{i=1}^n P(x_n|S)}{P(S)\prod_{i=1}^n P(x_n|S) + P(H)\prod_{i=1}^n P(x_n|H)} $$

In [99]:
def msg_lower(msg_list):
    '''
    Converts all characters in a list of messages
    to lower case. Conversion is performed in-place.
    '''
    for i in range(0,len(msg_list)):
        msg_list[i] = msg_list[i].lower()
    
    return None
    
    
def word_set(msg_list):
    '''
    Function to create set of unique words from 
    list of messages.
    Returns a set of strings.
    '''
    words = set()
    for msg in msg_list:
        for word in msg.split():
            words.add(word)
    
    return words


def word_smooth_prob(word, msg_list):
    '''
    Calculate the Laplace smoothed probability of word appearing 
    in each message in msg_list.
    1. Count how many messages in msg_list contain word & add 1. 
    2. Divide this by the number of messages in msg_list plus 2.
    Returns a float.
    '''
    count = 0
    for msg in msg_list:
        if word in msg.split():
            count += 1
    
    return (float(count) + 1) / (float(len(msg_list)) + 2)


def word_prob_dict(word_list, msg_list):
    '''
    Returns a dictionary of word:probability pairs for the
    words in the input word_list.
    '''
    prob_dict = {}
    for word in word_list:
        prob_dict[word.lower()] = word_smooth_prob(word.lower(), msg_list)
        
    return prob_dict

def marginal_prob(a_list,b_list):
    '''
    '''
    return len(a_list) / (len(a_list) + len(b_list))

In [102]:
# Convert all words in message lists to lower case before matching
msg_lower(previous_spam)
msg_lower(previous_ham)

# Create sets of unique words per message list
previous_spam_words = word_set(previous_spam)
previous_ham_words = word_set(previous_ham)

# Build dictionaries of probabilities for each word per message list
previous_spam_prob = word_prob_dict(word_set(previous_spam), previous_spam)
previous_ham_prob = word_prob_dict(word_set(previous_ham), previous_ham)

# Calculate the marginal probabilities for spam & ham
marg_prob_spam = marginal_prob(previous_spam, previous_ham)
marg_prob_ham = marginal_prob(previous_ham, previous_spam)

<hr style=\"border:2px solid gray\"> </hr>

## Reflection

Write you reflection in below cell.